# Basic AI Object Detection

## Your First Real-Time AI Experience

Now you'll see AI object detection in action. Your IMX500 camera will identify objects in real-time and draw bounding boxes around them with confidence scores.

## What You'll Observe

**Bounding Boxes**: Colored rectangles around detected objects showing their location

**Object Labels**: Text identifying what the AI recognizes (person, cell phone, cup, etc.)

**Confidence Scores**: Percentages showing how certain the AI is about each detection

**Real-time Processing**: All analysis happens at 30 frames per second on the camera sensor

## Understanding the Results

The AI has been trained on 80 different object types. It doesn't just guess - it calculates probabilities and only shows detections above a confidence threshold.

When you see "person 87%", the AI is saying there's an 87% chance that object is a person based on the visual patterns it learned from millions of training images.

## Live AI Object Detection

Run the cell below to start real-time AI object detection. Try pointing the camera at different objects: phones, cups, books, your hand, or any household items.

## AI Detection Stream

Run the cell below to start your live AI object detection feed:

In [ ]:
import subprocess
import cv2
import numpy as np
from IPython.display import display, clear_output, Image
import ipywidgets as widgets
import threading
import time

# Stop button in notebook
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop AI Detection',
    button_style='danger',
    icon='square'
)

display(stopButton)

def ai_detection_stream():
    # Use libcamera-vid with AI post-processing to stream MJPEG with detections
    cmd = [
        "libcamera-vid",
        "--inline",           # Needed for streaming
        "-t", "0",            # No timeout
        "--width", "640",
        "--height", "480",
        "--codec", "mjpeg",
        "--post-process-file", "/usr/share/rpi-camera-assets/imx500_mobilenet_ssd.json",
        "-o", "-"             # Output to stdout
    ]
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    data = b""
    display_handle = display(None, display_id=True)

    while not stopButton.value:
        chunk = proc.stdout.read(1024)
        if not chunk:
            break
        data += chunk
        a = data.find(b'\xff\xd8')  # JPEG start
        b = data.find(b'\xff\xd9')  # JPEG end
        if a != -1 and b != -1:
            jpg = data[a:b+2]
            data = data[b+2:]
            frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            if frame is not None:
                _, jpeg = cv2.imencode('.jpg', frame)
                display_handle.update(Image(data=jpeg.tobytes()))
    proc.terminate()

# Start AI detection stream in a separate thread
threading.Thread(target=ai_detection_stream, daemon=True).start()

print("AI Object Detection is now running!")
print("You should see bounding boxes around detected objects with labels and confidence scores.")
print("Try showing the camera: phones, cups, books, your hand, or any objects nearby.")
print("Click 'Stop AI Detection' when finished.")

## Understanding What You're Seeing

**Bounding Boxes**: The colored rectangles show where the AI detected objects. Each box precisely outlines the detected item.

**Labels**: Text above each box shows what the AI recognized. The system can identify 80 different object types.

**Confidence Scores**: The percentages indicate how certain the AI is. Higher numbers mean more reliable detections.

**Real-time Processing**: All analysis happens on the IMX500 sensor at 30 frames per second - no delays or lag.

## Try This Experiment

While the detection is running, try these tests:

1. **Move objects closer and farther** - notice how confidence changes
2. **Change lighting** - see how it affects detection accuracy  
3. **Partial occlusion** - cover half an object and see if it's still detected
4. **Multiple objects** - show several items at once
5. **Different angles** - rotate objects and observe the results

These experiments help you understand AI strengths and limitations.

## Detectable Objects

Below is a list of all of the pre-trained objects the camera will detect: